In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris

In [12]:
# Matrix Multiplication of Feature Matrix and Weight Matrix
class Matrix_Multiplication:
    def __init__(self,d,N):
        self.N = N
        self.W = np.zeros([d,N])
        self.X = 0
    def ForwardPass(self,X):
        self.X = X
        return np.dot(X,self.W)
    def BackwardPass(self,grad):
        dw,dx = np.dot(self.X.T,grad),np.dot(grad,self.W.T) 
        self.W = self.W - 0.000001*dw
        return dx

# Bias Addition Layer to the Neural Network   
class Bias_Layer:
    def __init__(self,N):
        self.Y = 0
        self.B = np.array([np.ones(N)])
        self.N = N
    def ForwardPass(self,y):
        self.Y = y
        T = y.T
        L = []
        for i in range(self.N):
            L.append(T[i]+self.B[0][i])
        return np.array(L).T

    def BackwardPass(self,grad):
        dN,dB = grad,sum(grad)
        self.B = self.B - 0.000001*dB
        return dN

# Finding Mean Squared Error   
class Mean_Square_Loss:
    def __init__(self):
        self.Y = 0
        self.P = 0
    def ForwardPass(self,p,y):
        self.P = p
        self.Y = y
        return sum((p-y)**2)
    def BackwardPass(self):
        return 2*(self.P - self.Y)

# Creating the Softmax Layer, Probability Layer    
class Soft_Max:
    def __init__(self):
        self.Z = 0
    def ForwardPass(self,z):
        self.Z = z
        self.L = 0
        L = []
        for j in range(len(z)):
            L.append([])
            for i in range(len(self.Z[j])):
                Nemo = np.exp(z[j][i])
                Demo = sum(np.exp(z[j]))
                L[j].append(Nemo/Demo)
        L = np.array(L)
        self.L= L
        return L
    def BackwardPass(self,out):
        L = []
        for i in range(len(out)):
            A = np.zeros([len(self.Z[0]),len(self.Z[0])])
            S = self.L
            for j in range(len(self.Z[0])):
                for k in range(len(self.Z[0])):
                    if j == k:
                        A[j][k] = S[i][k]*(1-S[i][k])
                    else:
                        A[j][k] = -S[i][j]*S[i][k]
            L.append(np.dot(out[i],A))
        return np.array(L)

# Sigmoid ActivationFuncLayer Function
class Sigmoid:
    def __init__(self):
        self.Z = 0
        self.O = 0
    def ForwardPass(self,z):
        self.Z = z
        self.O = 1/(1+np.exp(-self.z))
        return self.O
    def BackwardPass(self,grad):
        print('Back Input to Sigmoid = ',grad)
        S = self.O
        return (S*(S-1))*grad

# Cross Entropy Loss Function
class Cross_Entropy_Loss:
    def __init__(self):
        self.Y = 0
        self.P = 0
    def ForwardPass(self,p,y):
        self.P = p
        self.Y = [y]
        L = []
        for i in self.P:
            L.append(-np.log(i[y]))
            # print(i[y])
        return sum(L)
    def BackwardPass(self):
        dL = np.zeros([self.P.shape[0],self.P.shape[1]])
        Y =  np.zeros([self.P.shape[0],self.P.shape[1]])
        for i in range(len(self.Y)):
            Y[i][self.Y[i]] = 1
        for i in range(len(dL)):
            for j in range(len(dL[0])):
                dL[i][j] = -Y[i][j]/self.P[i][j]
        return dL

In [13]:
## Loading the Iris Data Set
Data_Iris = load_iris()

In [14]:
# Converting the Data Set into a Pandas Data Frame
iris  = pd.DataFrame(Data_Iris.data, columns=Data_Iris.feature_names)
Y = Data_Iris.target
iris = iris.drop(['petal width (cm)'],axis=1)
X = pd.DataFrame.to_numpy(iris)
X = list(map(list,list(X)))

In [15]:
Obj1 = Matrix_Multiplication(len(X[0]),3)
Obj2 = Bias_Layer(3)
Obj3 = Soft_Max()
Obj4 = Cross_Entropy_Loss()
itr = 0
while( itr < 1000):
    for i in range(len(X)):
        # forwardpropagation
        Out1 = Obj1.ForwardPass(np.array([X[i]]))
        Out2 = Obj2.ForwardPass(Out1)
        Out3 = Obj3.ForwardPass(Out2)
        Out4 = Obj4.ForwardPass(Out3,Y[i])
        # backpropagation
        dy1 =  Obj4.BackwardPass()
        dy2 =  Obj3.BackwardPass(dy1)
        dy3 =  Obj2.BackwardPass(dy2)
        dy4 =  Obj1.BackwardPass(dy3)
    itr+=1

In [16]:
W = Obj1.W
B = Obj2.B
L = []
for i in range(len(X)):
    Soft2 = Soft_Max()
    Out = Soft2.ForwardPass((np.dot(np.array([X[i]]),W).T+B.T).T)
    Out = list(Out[0])
    L.append(Out.index(max(Out)))

In [17]:
errorCnt =0 
for i in range(len(Y)):
    if Y[i] != L[i]:
        errorCnt+=1
print('Count of Wrong Prediction :',errorCnt)

Count of Wrong Prediction : 76
